<a href="https://colab.research.google.com/github/mcregragui/Solveur-lin-aire/blob/master/IRM_simul.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Projet IRM simulation:***




*Librairies nécessaires:*

In [112]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure, plot, subplot, title, xlabel, ylabel
from IPython.display import Markdown as md
import math as m
from numpy import mean, std, exp, diag, matmul, pi, cos, sin, zeros, ones, shape, floor, ceil, sqrt
from IPython.display import display
import ipywidgets as widgets
from matplotlib.widgets import Slider


# **Partie 1:**

On va définir dans le bolc suivant des fonctions auxiliares :


1.   Les fonctions de rotations xrot, yrot qui expriment l'excitation de signal.
2.   La fonction de rotation suivant z de precession.
3.   La fonction freeprecess exprime la relaxation longitidunale: $M_{xy}\exp(-t/T_1)$, et la relaxation transversale: $M_z\exp(-t/T_2)$.





---



In [49]:
##1

def xrot(angle=0.,in_deg=False):
  if in_deg:
    angle=angle*np.pi/180.
  c=np.cos(angle)
  s=np.sin(angle)
  M=np.array([[1.,0.,0.],[0.,c,-s],[0.,s,c]])
  return M

def yrot(angle=0.,in_deg=False):
  if in_deg:
    angle=angle*np.pi/180.
  c=np.cos(angle)
  s=np.sin(angle)
  M=np.array([[0.,c,-s],[1.,0.,0.],[0.,s,c]])
  return M

def zrot(angle=0.,in_deg=False):
  if in_deg:
    angle=angle*np.pi/180.
  c=np.cos(angle)
  s=np.sin(angle)
  M=np.array([[0.,c,-s],[0.,s,c],[1.,0.,0.]])
  return M

##2

def throt(phi=0.,theta=0.,in_deg=True):
  if in_deg:
    theta=theta*np.pi/180.
    phi=phi*np.pi/180.

def freeprecess(t,T1,T2,df):
    phi=2*np.pi*df*t/1000
    E1=np.exp(-t/T1)
    E2=np.exp(-t/T2)
    A=np.diag([E2,E2,E1])@zrot(phi,False)
    B=np.array([0,0,1-E1])

    B=np.reshape(B,[3,1])
    return A,B  


Spoiler RF:
  
*   Si on veut réduire TR afin de trouver l'image plus vite, alors il sera mieux de détruire la partie transversale. Cette méthode est faite pour ce cas.
*   En effet si on veut négliger directement la partie transversale, on doit avoir un TR très long.






In [62]:
def fsesignal(T1,T2,TE,TR,dfreq,angle,nb):
    Rflip = yrot(angle) # Rotation suivant y.
    dT=1 #pas de temps
    ceil=round(TR/dT)
    M=np.zeros((3,nb*ceil))
    [A1,B1] = freeprecess(dT,T1,T2,dfreq)
    M[:,0]=np.array([0,0,1])
    cpt=1
    
    for k in range(nb):
      M[:,cpt]=Rflip@M[:,cpt]
      for i in range(ceil):
        cpt+=1
        M[:,cpt:cpt+1]=A1@M[:,cpt-1:cpt]+B1
    return M


Les graphes de M suivant x, y et z.

In [182]:
dfreq=1.
@widgets.interact(TR=(0, 1000, 10), TE=(0, 500, 10), angle=(0, 2*np.pi, 0.01),nb=(0, 100, 1),T1=(0, 1000, 10),T2=(0, 1000, 10))
def update(TR = 100, TE=10, angle=np.pi/3,nb=10,T1=10,T2=10):
#ceil=round(TR/dT)
  tm=TR*nb
  time = np.arange(0, tm, 1)
  M=fsesignal(T1,T2,TE,TR,dfreq,angle,nb)

  plt.plot(time,M[0,:], color='blue')
  plt.title("Mx")


interactive(children=(IntSlider(value=100, description='TR', max=1000, step=10), IntSlider(value=10, descripti…

In [183]:
@widgets.interact(TR=(0, 1000, 10), TE=(0, 500, 10), angle=(0, 2*np.pi, 0.01),nb=(0, 100, 1),T1=(0, 1000, 10),T2=(0, 1000, 10))
def update(TR = 100, TE=10, angle=np.pi/3,nb=10,T1=10,T2=10):
#ceil=round(TR/dT)
  tm=TR*nb
  time = np.arange(0, tm, 1)
  M=fsesignal(T1,T2,TE,TR,dfreq,angle,nb)

  plt.plot(time,M[1,:], color='orange')
  plt.title("My")

interactive(children=(IntSlider(value=100, description='TR', max=1000, step=10), IntSlider(value=10, descripti…

In [184]:
@widgets.interact(TR=(0, 1000, 10), TE=(0, 500, 10), angle=(0, 2*np.pi, 0.01),nb=(0, 100, 1),T1=(0, 1000, 10),T2=(0, 1000, 10))
def update(TR = 100, TE=10, angle=np.pi/3,nb=10,T1=10,T2=10):
  tm=TR*nb
  time = np.arange(0, tm, 1)
  M=fsesignal(T1,T2,TE,TR,dfreq,angle,nb)
  plt.plot(time,M[2,:], color='green')
  plt.title("Mz")

interactive(children=(IntSlider(value=100, description='TR', max=1000, step=10), IntSlider(value=10, descripti…

Spoiler *parfait*


*   On veut que la partie transversale soit nule: $M_{xy}\sim0$




In [98]:
 def par_fsesignal(T1,T2,TE,TR,dfreq,angle,nb):
    Rflip = yrot(angle) # Rotation from excitation pulse (90)
    dT=1
    ceil=round(TR/dT)
    M=np.zeros((3,nb*ceil))
    [A1,B1] = freeprecess(dT,T1,T2,dfreq)
    M[:,0]=np.array([0,0,1])
    cpt=1
    
    for k in range(nb):
      M[:,cpt]=Rflip@M[:,cpt]
      M[:,cpt][0]=0
      M[:,cpt][1]=0
      for i in range(ceil):
        cpt+=1
        M[:,cpt:cpt+1]=A1@M[:,cpt-1:cpt]+B1
        
    return M

La graphe de M suivant z:

In [100]:
dfreq=1.
@widgets.interact(TR=(0, 1000, 10), TE=(0, 500, 10), angle=(0, 2*np.pi, 0.01),nb=(0, 100, 1),T1=(0, 1000, 10),T2=(0, 1000, 10))
def update(TR = 100, TE=10, angle=np.pi/3,nb=10,T1=10,T2=10):
  tm=TR*nb
  time = np.arange(0, tm, 1)
  M=par_fsesignal(T1,T2,TE,TR,dfreq,angle,nb)

  plt.plot(time,M[2,:], color='blue')
  plt.title("Mz")


interactive(children=(IntSlider(value=100, description='TR', max=1000, step=10), IntSlider(value=10, descripti…

Intensité gradient

In [156]:
def pre_function(flip,T1,T2,TE,TR,dfreq,phi):
  Rflip = yrot(flip)
  [Atr,Btr] = freeprecess(TR-TE,T1,T2,dfreq)
  [Ate,Bte] = freeprecess(TE,T1,T2,dfreq)
  Atr = zrot(phi)@Atr
  Mss = np.linalg.inv(np.eye(3)-Ate@Rflip@Atr) @ (Ate@Rflip@Btr+Bte)
  Msig = Mss[0]+Mss[1]*1j
  return Msig,Mss

In [189]:
def function(flip,T1,T2,TE,TR,dfreq,N):
  
  M =np.zeros((3,N))
  phi=[]
  for k in range(N):
    g=float(k)
    t=float(N)
    phi+=[((k+1.)/N-0.5)*4*np.pi]

  for k in range(N):
    [M1sig,M1]=pre_function(flip,T1,T2,TE,TR,dfreq,phi[k])
    M[:,k:k+1]=M1
  return M.mean(0)
  

In [190]:
dfreq=1.
@widgets.interact(TR=(0, 1000, 10), TE=(0, 500, 10), flip=(0, 10*np.pi, np.pi/100),N=(0, 1000, 1),T1=(0, 1000, 10),T2=(0, 1000, 10))
def update(TR = 100, TE=10, flip=2*np.pi,N=100,T1=100,T2=50):
  time = np.arange(0, N, 1)
  M=function(flip,T1,T2,TE,TR,dfreq,N)

  plt.plot(time,M, color='blue')
  plt.title("Intensité spoiler gradient")


interactive(children=(IntSlider(value=100, description='TR', max=1000, step=10), IntSlider(value=10, descripti…

# **Partie 2:**

Echo spin

Dans cette partie on simule la formule suivante:
\begin{aligned}
S=& M_{0} \sin \theta_{1} \sin ^{2}\left(\frac{\theta_{2}}{2}\right) \frac{\left(1+\left(\cos \theta_{2}-1\right) e^{-(\mathrm{TR}-\mathrm{TE} / 2) / T_{1}}-\cos \theta_{2} e^{-\mathrm{TR} / T_{1}}\right)}{1-\cos \theta_{1} \cos \theta_{2} e^{-\mathrm{TR} / T_{1}}}\times e^{-\mathrm{TE} / T_{2}}
\end{aligned}


In [180]:
def S(TE,TR,T1,T2,theta1,theta2):
  a=np.sin(theta1)*np.sin(theta2/2)**2
  b=1-(np.cos(theta2)-1)*np.exp(-(TR-TE/2)/T1)-cos(theta2)*np.exp(-TR/T1)
  c=1-np.cos(theta1)*np.cos(theta2)*np.exp(-TR/T1)
  d=np.exp(-TR/T2)
  return a*d*b/c

In [188]:

dt=2
T1 = np.arange(dt,1000,dt)
T2=100

@widgets.interact(TR=(0, 10000, 10), TE=(0, 1000, 5), theta1=(0, 2*np.pi, 0.01),theta2=(0, 2*np.pi, 0.01))
def update(TR = 1000, TE=100, theta1=np.pi/3,theta2=np.pi/3):
    plt.plot(T1,S(TE,TR,T1,T2,theta1,theta2), color='blue')
    plt.show()

interactive(children=(IntSlider(value=1000, description='TR', max=10000, step=10), IntSlider(value=100, descri…